In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export

import json
from os import environ
from time import sleep
from nifi_api.dataflow import DataFlow
from nifi_api.environment import DataFlowIds

In [ ]:
#default_exp source_to_refined

# Source To Refined
> Monitors three dataflows

In [ ]:
#export
class NifiIds:
    """ The collection of the DataFlowIds:
          - source to raw
          - raw to discovery
          - discovery to refined
    """

    nifi_json_ids = json.load(open(environ['CONFIG_FILE']))['NifiIds']

    sourceToRaw = DataFlowIds(pipeline=nifi_json_ids['SourceToRaw'])
    rawToDiscovery = DataFlowIds(pipeline=nifi_json_ids['RawToDiscovery'])
    discoveryToRefined = DataFlowIds(
        pipeline=nifi_json_ids['DiscoveryToRefined'])

In [ ]:
# export
def source_to_refined():
    """ The Source To Refined dataflow is decomposed in three sequential steps:
        - Source To Raw
        - Raw To Discovery
        - Discovery To Refined
    This function assures that each of these steps starts only after the
    previous step has finished.
    """

    # time between steps
    T = 15
    time_after_start = 15

    time_between_checks = 15

    source_to_raw = DataFlow(
        NifiIds.sourceToRaw,
        delay_seconds_after_start=time_after_start,
        delay_seconds_between_checks=time_between_checks,
    )
    raw_to_discovery = DataFlow(
        NifiIds.rawToDiscovery,
        delay_seconds_after_start=time_after_start,
        delay_seconds_between_checks=time_between_checks,
    )
    discovery_to_refined = DataFlow(
        NifiIds.discoveryToRefined,
        delay_seconds_after_start=time_after_start,
        delay_seconds_between_checks=time_between_checks,
    )

    source_to_raw.run()
    sleep(T)
    raw_to_discovery.run()
    sleep(T)
    discovery_to_refined.run()

In [ ]:
#test

# Benchmark Test
# 1. Update a valid 1 GB file in to the S3 bucket, i.e., campaigns.csv
# 2. Run *source_to_refined* and track its execution time
source_to_refined()

Pipeline watching has finished ...
Pipeline watching has finished ...
Pipeline watching has finished ...


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_environment.ipynb.
Converted 02_rest.ipynb.
Converted 03_dataflow.ipynb.
Converted 04_source_to_refined.ipynb.
Converted 09_tools.ipynb.
Converted index.ipynb.
